In [5]:
from mlflow.tracking import MlflowClient
import mlflow

ML_FLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=ML_FLOW_TRACKING_URI)

In [42]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.test_rmse < 9',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_rmse ASC"]
)

In [43]:
runs

[<Run: data=<RunData: metrics={'test_rmse': 8.645136751841473, 'val_rmse': 8.105536545425169}, params={'Test-data_path': 'data/chicago_taxi_test_dataset_2023-03.parquet',
  'Validation-data_path': 'data/chicago_taxi_Val_dataset_2023-02.parquet',
  'learning_rate': '0.5458691870793922',
  'max_depth': '38',
  'min_child_weight': '7.968333201830882',
  'objective': 'reg:linear',
  'reg_alpha': '0.19154033738324955',
  'reg_lambda': '0.032825863134369984',
  'seed': '42',
  'train-data_path': 'data/chicago_taxi_train_dataset_2023-01.parquet'}, tags={'developer': 'Arun',
  'mlflow.log-model.history': '[{"run_id": "aea032c047344461a62915b096cca1ee", '
                              '"artifact_path": "models", "utc_time_created": '
                              '"2023-08-10 23:56:21.711254", "flavors": '
                              '{"python_function": {"loader_module": '
                              '"mlflow.xgboost", "python_version": "3.9.17", '
                              '"data": "m

In [44]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['test_rmse']:.3f}")

run id: aea032c047344461a62915b096cca1ee, rmse: 8.645
run id: 39a811d73b284f92a656d103242c70e4, rmse: 8.658
run id: 4033a44dd181406b8dd876faa49a3170, rmse: 8.666


In [45]:
mlflow.set_tracking_uri(ML_FLOW_TRACKING_URI)

In [46]:
run_id = "aea032c047344461a62915b096cca1ee"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="chicago-taxi-prediction-model")

Successfully registered model 'chicago-taxi-prediction-model'.
2023/08/10 23:01:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: chicago-taxi-prediction-model, version 1
Created version '1' of model 'chicago-taxi-prediction-model'.


<ModelVersion: aliases=[], creation_timestamp=1691722895624, current_stage='None', description=None, last_updated_timestamp=1691722895624, name='chicago-taxi-prediction-model', run_id='aea032c047344461a62915b096cca1ee', run_link=None, source='/home/arundac23/mlops-2023-project/experiment_tracking/mlruns/1/aea032c047344461a62915b096cca1ee/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [47]:
run_id = "39a811d73b284f92a656d103242c70e4"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="chicago-taxi-prediction-model")

Registered model 'chicago-taxi-prediction-model' already exists. Creating a new version of this model...
2023/08/10 23:04:12 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: chicago-taxi-prediction-model, version 2
Created version '2' of model 'chicago-taxi-prediction-model'.


<ModelVersion: aliases=[], creation_timestamp=1691723052799, current_stage='None', description=None, last_updated_timestamp=1691723052799, name='chicago-taxi-prediction-model', run_id='39a811d73b284f92a656d103242c70e4', run_link=None, source='/home/arundac23/mlops-2023-project/experiment_tracking/mlruns/1/39a811d73b284f92a656d103242c70e4/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [48]:
run_id = "4033a44dd181406b8dd876faa49a3170"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="chicago-taxi-prediction-model")

Registered model 'chicago-taxi-prediction-model' already exists. Creating a new version of this model...
2023/08/10 23:05:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: chicago-taxi-prediction-model, version 3
Created version '3' of model 'chicago-taxi-prediction-model'.


<ModelVersion: aliases=[], creation_timestamp=1691723144657, current_stage='None', description=None, last_updated_timestamp=1691723144657, name='chicago-taxi-prediction-model', run_id='4033a44dd181406b8dd876faa49a3170', run_link=None, source='/home/arundac23/mlops-2023-project/experiment_tracking/mlruns/1/4033a44dd181406b8dd876faa49a3170/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [61]:
model_name = 'chicago-taxi-prediction-model'
latest_version = client.get_latest_versions(name=model_name)
for version in latest_version:
    print(f"version: {version.version}, status: {version.current_stage}")

version: 3, status: None


In [65]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="staging",
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1691722895624, current_stage='Staging', description=None, last_updated_timestamp=1691724430158, name='chicago-taxi-prediction-model', run_id='aea032c047344461a62915b096cca1ee', run_link=None, source='/home/arundac23/mlops-2023-project/experiment_tracking/mlruns/1/aea032c047344461a62915b096cca1ee/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [67]:
from datetime import datetime
date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
client.update_model_version(
    name=model_name,
    version=1,
    description=f"Moved version 1 is selected for staging for this {model_name} at {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1691722895624, current_stage='Staging', description=('Moved version 1 is selected for staging for this '
 'chicago-taxi-prediction-model at 2023-08-10 23:29:19'), last_updated_timestamp=1691724559537, name='chicago-taxi-prediction-model', run_id='aea032c047344461a62915b096cca1ee', run_link=None, source='/home/arundac23/mlops-2023-project/experiment_tracking/mlruns/1/aea032c047344461a62915b096cca1ee/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>